In [1]:
pip install sklearn tensorflow matplotlib numpy

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.2.9/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [342]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np

In [363]:
enc = preprocessing.LabelEncoder()
df = pd.read_csv('newCombined.csv')
dg = pd.read_csv('wData.csv')
dFF = dg['DATE']
scaler = MinMaxScaler()

In [364]:
floatPCRP = dg['PRCP'].values.astype(float)
scaledPCRP = scaler.fit_transform(floatPCRP.reshape(-1,1))
floatTMAX = dg['TMAX'].values.astype(float)
scaledTMAX = scaler.fit_transform(floatTMAX.reshape(-1,1))
floatTMIN = dg['TMIN'].values.astype(float)
scaledTMIN = scaler.fit_transform(floatTMIN.reshape(-1,1))
floatSM = dg['SM 4in'].values.astype(float)
scaledSM = scaler.fit_transform(floatSM.reshape(-1,1))
floatHM = dg['Relative Humidity (pct) Mean of Hourly Values'].values.astype(float)
scaledHM = scaler.fit_transform(floatHM.reshape(-1,1))
floatWS = dg['Wind Speed Average (mph) Mean of Hourly Values'].values.astype(float)
scaledWS = scaler.fit_transform(floatWS.reshape(-1,1))
#floatTOT = df['total'].values.astype(float)
#scaledTOT = scaler2.fit_transform(floatTOT.reshape(-1,1))
#scaledTOT = scaler.fit_transform(floatTOT.reshape(-1,1))




dg['PRCP'] = pd.DataFrame(scaledPCRP)
dg['TMAX'] = pd.DataFrame(scaledTMAX)
dg['TMIN'] = pd.DataFrame(scaledTMIN)
dg['SM 4in'] = pd.DataFrame(scaledSM)
dg['Relative Humidity (pct) Mean of Hourly Values'] = pd.DataFrame(scaledHM)
dg['Wind Speed Average (mph) Mean of Hourly Values'] = pd.DataFrame(scaledWS)
#df['total'] = pd.DataFrame(scaledTOT)
df['total'] = pd.cut(df.total,bins=[0,1000,5000,20000,50000],labels=['Low Risk','Moderate','High Risk','Extreme Risk'])
#df.total[24] = "Low Risk"
df['total'] = enc.fit_transform(df['total'])

print(df.total)
#print(scaler.inverse_transform(df.total.values.reshape(-1,1)))

0     3
1     1
2     3
3     3
4     3
5     1
6     1
7     1
8     3
9     1
10    3
11    1
12    1
13    1
14    1
15    1
16    1
17    0
18    1
19    3
20    3
21    1
22    2
23    2
24    2
25    3
26    1
27    3
28    3
29    3
30    1
31    3
32    1
33    3
34    3
35    3
36    3
37    3
38    3
39    3
40    3
41    2
42    1
43    3
44    3
45    3
46    1
47    0
48    3
49    1
50    2
51    1
52    3
53    1
Name: total, dtype: int64


In [426]:
xData =[]
dateAtIndex = df['Date']
dateAtIndex = dateAtIndex.tolist()
for i in range(len(df)):
    
    indexOfDate = dFF.tolist().index(dateAtIndex[i])
        # xData= np.append(xData, dg[i-6+28:i+1+28].to_numpy())
    #print(dg[i-6+28:i+1+28].to_numpy())
    xData.append(dg[indexOfDate-13:indexOfDate+1].drop(['DATE'], axis = 1).to_numpy())
xData = np.array(xData)
#xData = xData.reshape(xData.shape[0], xData.shape[2] * xData.shape[1])
#xData = xData.reshape(xData.shape[0], xData.shape[1], 1)
print(xData.shape)


(54, 14, 6)


In [427]:
xData = xData.reshape(xData.shape[0], xData.shape[2] * xData.shape[1])

In [428]:
xTrain = xData[:41]

ySet = df['total'].values

yTrain = ySet[:41]
xTest = xData[41:]
yTest = ySet[41:]
print(xTrain)

[[0.09208401 0.73660714 0.81060606 ... 0.84452297 0.95698925 0.24193548]
 [0.27140549 0.75446429 0.84848485 ... 0.86925795 0.86021505 0.29032258]
 [0.         0.75892857 0.87121212 ... 0.74204947 0.82795699 0.28225806]
 ...
 [0.         0.80803571 0.91666667 ... 0.87632509 0.86021505 0.10483871]
 [0.01938611 0.8125     0.93560606 ... 0.82332155 0.87096774 0.10483871]
 [0.273021   0.83035714 0.88257576 ... 0.74911661 0.8172043  0.16935484]]


In [429]:
# model = Sequential()
# model.add(LSTM(200, input_shape=[11,6]))
# #model.add(Dense(100, input_shape=[42]))
# model.add(Dense(42, activation='sigmoid'))
# # model.add(Dense(4, activation='relu'))
# model.add(Dense(1))
# model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])


model = Sequential()
model.add(Dense(200, input_shape=[xTrain.shape[1],]))
#model.add(Dense(100, input_shape=[42]))
# model.add(Dense(42, activation='sigmoid'))
# model.add(Dense(4, activation='relu'))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [430]:
model.fit(xTrain, yTrain, epochs=50, batch_size=6)

Epoch 1/50
7/7 [==============================] - 0s 2ms/step - loss: 2.7946 - accuracy: 0.3415
Epoch 2/50
7/7 [==============================] - 0s 2ms/step - loss: 1.0918 - accuracy: 0.3902
Epoch 3/50
7/7 [==============================] - 0s 3ms/step - loss: 0.9845 - accuracy: 0.3902
Epoch 4/50
7/7 [==============================] - 0s 2ms/step - loss: 1.2661 - accuracy: 0.3902
Epoch 5/50
7/7 [==============================] - 0s 2ms/step - loss: 0.9877 - accuracy: 0.3902
Epoch 6/50
7/7 [==============================] - 0s 2ms/step - loss: 1.0081 - accuracy: 0.3902
Epoch 7/50
7/7 [==============================] - 0s 2ms/step - loss: 0.8563 - accuracy: 0.3902
Epoch 8/50
7/7 [==============================] - 0s 2ms/step - loss: 0.9810 - accuracy: 0.3902
Epoch 9/50
7/7 [==============================] - 0s 3ms/step - loss: 0.9790 - accuracy: 0.3902
Epoch 10/50
7/7 [==============================] - 0s 2ms/step - loss: 0.8077 - accuracy: 0.3902
Epoch 11/50
7/7 [======================

In [434]:
result = model.predict(xData, verbose=0)
# print(scaler.inverse_transform(result))
# print(scaler.inverse_transform(ySet.reshape(-1,1)))

[2.244345]


In [357]:
'''
[[2.2612743 2.212653  2.1985948 2.1955667 2.2299922 2.2387938 2.2102914]
 [2.3018181 2.2645514 2.2502685 2.2524965 2.2719753 2.2737863 2.2523875]
 [2.2605417 2.223388  2.2104933 2.216233  2.227001  2.2309234 2.218105 ]
 [2.2897768 2.2478154 2.235794  2.2450972 2.2553353 2.2559464 2.2453423]
 [2.4125617 2.3392959 2.335546  2.3550682 2.3818533 2.3791378 2.3416345]
 [2.243344  2.1933699 2.1822639 2.210369  2.2138507 2.2196887 2.1969297]
 [2.1836653 2.1375332 2.1240757 2.1487308 2.1522355 2.1593883 2.1413324]
 [2.1266723 2.1097176 2.0980394 2.0691657 2.0934727 2.1152477 2.1002758]
 [2.5192056 2.4647944 2.470552  2.4523406 2.5037434 2.493469  2.4554503]
 [2.540644  2.4768336 2.48342   2.4797966 2.530187  2.5239947 2.4713223]
 [2.513778  2.4518409 2.4566379 2.4641256 2.5055153 2.5052884 2.4492414]
 [1.6019405 1.6274765 1.6456753 1.6182985 1.6393309 1.6689913 1.6367085]
 [1.6398822 1.6211977 1.6328677 1.6466503 1.6533837 1.6655486 1.6470315]]
 '''

'\n[[2.2612743 2.212653  2.1985948 2.1955667 2.2299922 2.2387938 2.2102914]\n [2.3018181 2.2645514 2.2502685 2.2524965 2.2719753 2.2737863 2.2523875]\n [2.2605417 2.223388  2.2104933 2.216233  2.227001  2.2309234 2.218105 ]\n [2.2897768 2.2478154 2.235794  2.2450972 2.2553353 2.2559464 2.2453423]\n [2.4125617 2.3392959 2.335546  2.3550682 2.3818533 2.3791378 2.3416345]\n [2.243344  2.1933699 2.1822639 2.210369  2.2138507 2.2196887 2.1969297]\n [2.1836653 2.1375332 2.1240757 2.1487308 2.1522355 2.1593883 2.1413324]\n [2.1266723 2.1097176 2.0980394 2.0691657 2.0934727 2.1152477 2.1002758]\n [2.5192056 2.4647944 2.470552  2.4523406 2.5037434 2.493469  2.4554503]\n [2.540644  2.4768336 2.48342   2.4797966 2.530187  2.5239947 2.4713223]\n [2.513778  2.4518409 2.4566379 2.4641256 2.5055153 2.5052884 2.4492414]\n [1.6019405 1.6274765 1.6456753 1.6182985 1.6393309 1.6689913 1.6367085]\n [1.6398822 1.6211977 1.6328677 1.6466503 1.6533837 1.6655486 1.6470315]]\n '